###### **Parameters**

In [ ]:
# Input values here
source_system: str = ""
table_name: str = ""
endpoint: str = ""
http_method: str = ""
watermark_column: str = ""
max_watermark_cutoff: str = "" # fmt: '2023-12-01T00:00:00Z'
page_size: int = 500
query_template: str = ""
headers_json: str = ""

###### **Dynamic Path Resolution**

In [ ]:
ctx = notebookutils.runtime.context
wsid = ctx.get("currentWorkspaceId")

if not wsid: raise ValueError("Missing `currentWorkspaceId` from notebook runtime context.")

try:
    ops_artifact = notebookutils.lakehouse.get(name="Ops", workspaceId=wsid)
except Exception as e:
    raise RuntimeError(f"Could not return `Ops` lakehouse in this workspace ({wsid=}). Error: {repr(e)}") from e

if ops_artifact is None: raise ValueError("Ops artifact is None")

props = ops_artifact.get("properties") or {}
abfs = props.get("abfsPath")

if not abfs: raise ValueError("Ops artifact missing `properties.abfsPath`")

rest_collection_path = f"{abfs}/Tables/ops/rest_collection"

In [ ]:
# Imports
from delta.tables import DeltaTable
from pyspark.sql import DataFrame, functions as F, types as T
from datetime import datetime, timezone

def _rest_collection_schema() -> T.StructType:
    return T.StructType(
        [
            T.StructField("source_system", T.StringType(), nullable=False),
            T.StructField("table_name", T.StringType(), nullable=False),
            T.StructField("endpoint", T.StringType(), nullable=False),
            T.StructField("http_method", T.StringType(), nullable=False),
            T.StructField("watermark_column", T.StringType(), nullable=True),
            T.StructField("max_watermark_cutoff", T.TimestampType(), nullable=True),
            T.StructField("page_size", T.IntegerType(), nullable=True),
            T.StructField("query_template", T.StringType(), nullable=True),
            T.StructField("headers_json", T.StringType(), nullable=True),
        ]
    )

# convert string format to timestamp
max_watermark_cutoff = datetime.strptime(max_watermark_cutoff, "%Y-%m-%dT%H:%M:%SZ")

# Single-row source DF (the new/updated config row)
src_df = spark.createDataFrame([{
    "source_system": source_system,
    "table_name": table_name,
    "endpoint": endpoint,
    "http_method": http_method,
    "watermark_column": watermark_column,
    "max_watermark_cutoff": max_watermark_cutoff,
    "page_size": page_size,
    "query_template": query_template,
    "headers_json": headers_json,
}], schema=_rest_collection_schema())


display(src_df) # preview row to be added

###### **Merge Data**

In [ ]:
# Target delta table at ABFSS path
tgt = DeltaTable.forPath(spark, rest_collection_path)

(
    tgt.alias("t")
       .merge(
            src_df.alias("s"),
            "t.source_system = s.source_system AND t.table_name = s.table_name"
        )
       .whenMatchedUpdate(set={
            "endpoint": "s.endpoint",
            "http_method": "s.http_method",
            "watermark_column": "s.watermark_column",
            "max_watermark_cutoff": "s.max_watermark_cutoff",
            "page_size": "s.page_size",
            "query_template": "s.query_template",
            "headers_json": "s.headers_json",
        })
       .whenNotMatchedInsertAll()
       .execute()
)

###### **Preview Table**

In [ ]:
preview = True

if not preview: notebookutils.notebook.exit("")

df = spark.read.format("delta").load(rest_collection_path)
display(df)